In [ ]:
# Setup
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import psycopg2

DB_HOST = os.getenv('ANALYTICS_DB_HOST', 'localhost')
DB_PORT = int(os.getenv('ANALYTICS_DB_PORT', '5433'))
DB_NAME = os.getenv('ANALYTICS_DB_NAME', 'analytics_db')
DB_USER = os.getenv('ANALYTICS_DB_USER', 'postgres')
DB_PASSWORD = os.getenv('ANALYTICS_DB_PASSWORD', 'password')

conn = psycopg2.connect(host=DB_HOST, port=DB_PORT, dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD)
df = pd.read_sql_query("

SELECT
    sm.student_id,
    COALESCE(sm.engagement_score, 0) AS engagement_score,
    COALESCE(sm.success_rate, 0) AS success_rate,
    COALESCE(sm.punctuality_score, 0) AS punctuality_score,
    COALESCE(tf.engagement_ma30, 0) AS engagement_ma30,
    COALESCE(sm.login_frequency, 0) AS login_frequency,
    COALESCE(sm.forum_participation, 0) AS forum_participation
FROM student_metrics sm
LEFT JOIN temporal_features tf ON tf.student_id = sm.student_id;
"
, conn)
conn.close()
df.head()

In [ ]:
# PCA Projection
features = df[['engagement_score','success_rate','punctuality_score','engagement_ma30','login_frequency','forum_participation']].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)
plt.figure(figsize=(6,5))
plt.scatter(X_pca[:,0], X_pca[:,1], s=10, alpha=0.6)
plt.title('PCA Projection of Student Features')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()